<a href="https://colab.research.google.com/github/SahanaSwamy/Music-Genre-Recognition/blob/master/anysong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**This notebook is used to enter a song for the genre recognition model**

##**IMPORTING LIBRARIES**

In [ ]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
!pip install h5py pyyaml
!pip install tf_nightly

     |████████████████████████████████| 110.2MB 126kB/s 
     |████████████████████████████████| 512kB 39.4MB/s 
     |████████████████████████████████| 4.1MB 29.4MB/s 
     |████████████████████████████████| 61kB 18.6MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-2.3.2-cp36-none-any.whl size=49882 sha256=a6bc02cd2bc6e602de55d094c74b341e0b81c69afdc8e5375bf905771abacd3a
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


In [ ]:
#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!ls


sample_data


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
%cd /gdrive/My\ Drive/genres

/gdrive/My Drive/genres


In [ ]:
!ls

 anysong.ipynb	  hiphop	 newdata.csv	    scaler.pickle
 blues		  img_data	'newsong (1).mp3'   X1.pickle
 classical	  jazz		 newsong.mp3	    X2.pickle
 classical.mp3	  metal		 pop		    X.pickle
 country	  my_model.h5	 reggae		    y1.pickle
 data.csv	  newdata1.csv	 rock		    y2.pickle
 disco		  newdata2.csv	 rough1.ipynb
 FinalMus.model   newdata3.csv	 scaler


In [ ]:
##creating a spectogram for the one input song- passed as an mp3 file
cmap = plt.get_cmap('inferno')
import time
plt.figure(figsize=(10,10))
songname=f'/gdrive/My Drive/genres/classical.mp3'
y, sr = librosa.load(songname, mono=True, duration=5)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
time.sleep(1)
plt.savefig(f'img_data/example2')
plt.clf()

<Figure size 720x720 with 0 Axes>

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'

header = header.split()

## **CREATING A CSV**

In [ ]:
file = open('newdata3.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
y, sr = librosa.load(songname, mono=True, duration=30)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
# print(chroma_stft)
rmse = librosa.feature.rmse(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
songname=f'new'
to_append = f'{songname} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
for e in mfcc:
    to_append += f' {np.mean(e)}'
file = open('newdata3.csv', 'a', newline='')
with file:
  writer = csv.writer(file)
  writer.writerow(to_append.split())
            

In [ ]:
data=pd.read_csv('newdata3.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,new,0.246328,0.112484,558.318358,581.209446,891.477943,0.037158,-298.551732,196.988398,27.288655,-0.282789,4.734329,-6.352861,-5.152917,-7.108541,-12.305246,-13.802417,-15.335955,-15.268803,-12.98056,-9.619439,-7.790366,-6.414869,-2.911512,-4.147712,-9.525973,-9.198141


In [ ]:
data.shape

(1, 27)

In [ ]:
data = data.drop(['filename'],axis=1)

In [ ]:
data2=data

In [ ]:
data2

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.246328,0.112484,558.318358,581.209446,891.477943,0.037158,-298.551732,196.988398,27.288655,-0.282789,4.734329,-6.352861,-5.152917,-7.108541,-12.305246,-13.802417,-15.335955,-15.268803,-12.98056,-9.619439,-7.790366,-6.414869,-2.911512,-4.147712,-9.525973,-9.198141


In [ ]:
##import the scaler pickle object from the main notebook
import pickle
with open(r"scaler", "rb") as input_file:
  scaler = pickle.load(input_file)

In [ ]:
##transform the given song
X2 = scaler.transform(data)

In [ ]:
X2

array([[-1.62052818e+00, -2.80937152e-01, -2.29668578e+00,
        -3.15801354e+00, -2.33816089e+00, -1.58989455e+00,
        -1.53787238e+00,  3.11136384e+00,  1.66991000e+00,
        -2.19560707e+00,  4.81396765e-01, -1.77331276e+00,
        -2.34417438e-03, -1.64725624e+00, -6.41240367e-01,
        -2.71397137e+00, -1.36669159e+00, -2.94020634e+00,
        -1.32677534e+00, -2.27701687e+00, -8.04618641e-01,
        -1.65242609e+00,  2.32152961e-01, -1.20373142e+00,
        -1.91736186e+00, -2.11262325e+00]])

In [ ]:
##create an output variable y
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y2 = encoder.fit_transform(genre_list)

In [ ]:
y2

array([0])

In [ ]:
##load the model 
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0827 01:50:21.869843 140595759675264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 01:50:21.922616 140595759675264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 01:50:22.009242 140595759675264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0827 01:50:22.010473 140595759675264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0827 01:50:22.011807 140595759675264 depre

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               13824     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 186,954
Trainable params: 186,954
Non-trainable params: 0
_________________________________________________________________


##**PREDICTIONS**

In [ ]:
##pass your input song
predictions = new_model.predict_proba(X2)

In [ ]:
predictions

array([[5.9421211e-02, 3.5493079e-01, 5.4361767e-01, 8.4569736e-04,
        2.3032194e-06, 2.8121464e-02, 1.1234698e-08, 3.0554417e-03,
        1.4593394e-03, 8.5461261e-03]], dtype=float32)

In [ ]:
np.sum(predictions[0])

1.0000001

In [ ]:
np.max(predictions)

0.54361767

In [ ]:
np.argmax(predictions[0])

2

In [ ]:
s=np.argmax(predictions[0])

print(s)

if s==0:
  print("blues")
elif s==1:
  print("classical")
elif s==2:
  print("country")
elif s==3:
  print("disco")
elif s==4:
  print("hiphop")
elif s==5:
  print("jazz")
elif s==6:
  print("metal")
elif s==7:
  print("pop")
elif s==8:
  print("reggae")
else:
  print("rock")

2
country
